In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta
from pandas.tseries.offsets import BDay
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-03-03 2025-03-02


In [3]:
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(today, yesterday)

2025-03-03 2025-02-28


In [5]:
format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%','amt_yearly':'{:,.2f}','yearly':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [9]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}")

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Ad hoc
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### New dividend stock

In [6]:
name = '3BBIF'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [7]:
sqlIns = """
INSERT INTO dividend
VALUES('3BBIF',.16,0,0,0,.16,0,0,130000,'2024-02-21','2024-03-18','',1)
"""
rp = const.execute(sqlIns)
rp.rowcount

1

### Start of Update dividend

In [11]:
def update_dividend1(q4, XDATE, PAIDDATE, actual, name):
    # Use parameterized query to avoid SQL injection
    sql = text("""
        UPDATE dividend 
        SET q4 = :q4, 
            dividend = q1 + q2 + q3 + q4, 
            XDATE = :XDATE, 
            PAIDDATE = :PAIDDATE, 
            actual = :actual 
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sql, {
        'q4': q4,
        'XDATE': XDATE,
        'PAIDDATE': PAIDDATE,
        'actual': actual,
        'name': name
    })
    
    return f"Records updated = {rp.rowcount}"

In [13]:
name = 'SENA'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,SENA,0.2342,0.0000,0.0000,0.0000,0.2342,"105,000",2024-05-10,2024-05-24,,0


In [15]:
q4 = 0.111238
XDATE = '2025-05-09'
PAIDDATE = '2025-05-23'
actual = 1

update_dividend1(q4,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [17]:
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,SENA,0.1112,0.0000,0.0000,0.0000,0.1112,"105,000",2025-05-09,2025-05-23,,1


In [47]:
def update_dividend2(shares, q4, actual, name):
    # Use parameterized query to avoid SQL injection
    sql = text("""
        UPDATE dividend 
        SET q4 = :q4, 
            dividend = q1 + q2 + q3 + q4, 
            shares = :shares, 
            actual = :actual 
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sql, {
        'q4': q4,
        'shares': shares,
        'actual': actual,
        'name': name
    })
    
    return f"Records updated = {rp.rowcount}"

In [49]:
name = 'SCC'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,SCC,2.5000,0.0000,2.5000,0.0000,5.0000,600,2025-04-02,2025-04-22,,1


In [51]:
shares = 600
q4 = 2.50
actual = 1

In [53]:
update_dividend2(shares,q4,actual,name)

'Records updated = 1'

### Toggle actual status

In [ ]:
name = 'SENA'
actual = 0
sqlUpd = "UPDATE dividend SET actual = %s WHERE name = '%s'"
sqlUpd = sqlUpd % (actual, name)
rp = const.execute(sqlUpd)
rp.rowcount

### Delete dividend record

In [9]:
sqlDel = '''
DELETE FROM dividend
WHERE name IN ("BBL")
'''
rp = const.execute(sqlDel)
rp.rowcount

1

### Start of output to cloud

In [45]:
sql = """
SELECT Y.NAME AS name,  Q4 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND AS yearly, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND (Q4 > 0 OR Y.name IN ("GVREIT","TFFIF"))
AND P.date = '%s'
ORDER BY name
"""
sql = sql % today 
print(sql)


SELECT Y.NAME AS name,  Q4 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND AS yearly, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND (Q4 > 0 OR Y.name IN ("GVREIT","TFFIF"))
AND P.date = '2025-02-28'
ORDER BY name



In [47]:
cols = 'name qtrly shares amount net xdate paiddate cost_amt pct amt_yearly yearly actual'.split()

In [49]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['amt_yearly'] = round(dividend['shares'] * dividend['yearly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend['yearly'] = round(dividend['amt_yearly'] / dividend['cost_amt'] * 100, 2)
dividend[cols].style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,amt_yearly,yearly,actual
0,AH,0.4800,"1,200",576.00,518.40,2025-03-13,2025-05-23,"44,400.00",1.17%,936.00,2.11%,1
1,AIMIRT,0.2150,"12,500","2,687.50","2,418.75",2025-03-05,2025-03-21,"136,250.00",1.78%,"10,750.00",7.89%,1
2,ASP,0.1000,"30,000","3,000.00","2,700.00",2024-08-27,2024-09-12,"114,000.00",2.37%,"5,100.00",4.47%,0
3,AWC,0.0750,"9,000",675.00,607.50,2025-05-08,2025-05-28,"44,640.00",1.36%,675.00,1.51%,1
4,BCH,0.2500,"4,000","1,000.00",900.00,2024-08-29,2024-09-12,"86,800.00",1.04%,"1,480.00",1.71%,0
5,CPNREIT,0.0987,"55,000","5,428.50","4,885.65",2025-02-27,2025-03-17,"990,000.00",0.49%,"37,499.00",3.79%,1
6,DIF,0.2222,"35,000","7,777.00","6,999.30",2025-02-13,2025-03-07,"486,500.00",1.44%,"31,108.00",6.39%,1
7,GVREIT,0.2044,"75,000","15,330.00","13,797.00",2025-02-26,2025-03-12,"573,000.00",2.41%,"59,760.00",10.43%,1
8,IVL,0.1750,"7,200","1,260.00","1,134.00",2025-05-06,2025-05-23,"288,000.00",0.39%,"5,040.00",1.75%,1
9,JMT,0.2800,"4,200","1,176.00","1,058.40",2025-02-25,2025-04-30,"233,100.00",0.45%,"2,772.00",1.19%,1


In [51]:
formatted_amt = '{:,.2f}'.format(dividend.net.sum())
print(formatted_amt)

170,766.90


In [53]:
file_name = "dividend-q4.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [55]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\dividend-q4.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\dividend-q4.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\dividend-q4.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\dividend-q4.csv


In [59]:
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(god_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(osd_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(icd_file, index=False)

### End of output to cloud

In [ ]:
#sqlUpd = """
UPDATE dividend
SET xdate = DATE_ADD(xdate, INTERVAL 1 YEAR),
    paiddate = DATE_ADD(paiddate, INTERVAL 1 YEAR),
    actual = 0
WHERE Q4 > 0"""
rp = const.execute(sqlUpd)
rp.rowcount

In [49]:
const.commit()
const.close()